In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:2]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image

Image.open('/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/4ae939d7d.png')

In [ ]:
img1 = Image.open('/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/4ae939d7d.png')

img1_size = img1.size

print(img1_size)

In [ ]:
import glob

train = pd.DataFrame({'path' : glob.glob('/kaggle/input/plant-seedlings-classification/train/*/*')})
train

In [ ]:
train['target'] = train['path'].apply(lambda x:x.split('/')[-2])
train

In [ ]:
test = pd.DataFrame({'path': glob.glob('/kaggle/input/plant-seedlings-classification/test/*')})
test

In [ ]:
# from sklearn.model_selection import train_test_split

# x_train, x_valid = train_test_split(train, test_size = 0.2, 
#                                     random_state = 42, 
#                                     stratify = train['target'] )

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB1

# skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42 )

# result = 0

# for train_index, valid_index in skf.split(train, train['target']):
    
#     x_train = train.iloc[train_index]
#     x_valid = train.iloc[valid_index]
#     idg = ImageDataGenerator(horizontal_flip  = True)
#     idg2 = ImageDataGenerator()

#     train_gen = idg.flow_from_dataframe(x_train, x_col = 'path', y_col  = 'target', target_size = (100,100) )
#     valid_gen = idg2.flow_from_dataframe(x_valid, x_col = 'path', y_col  = 'target', target_size = (100,100) )
#     test_gen = idg2.flow_from_dataframe(test, x_col = 'path', y_col  = None,
#                                    class_mode = None,
#                                    shuffle = False,
#                                    target_size = (100,100) )
#     model = Sequential()
#     model.add(EfficientNetB1(include_top = False, pooling = 'avg'))
#     model.add(Dense(12, activation = 'softmax'))

#     model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
#          metrics = ['acc'])
#     model.fit(train_gen, epochs = 3,validation_data = valid_gen)
#     result += model.predict(test_gen, verbose = 1) / 3
    
    
    

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

idg = ImageDataGenerator(horizontal_flip  = True)
idg2 = ImageDataGenerator()

train_gen = idg.flow_from_dataframe(x_train, x_col = 'path', y_col  = 'target', target_size = (100,100) )
valid_gen = idg2.flow_from_dataframe(x_valid, x_col = 'path', y_col  = 'target', target_size = (100,100) )
test_gen = idg2.flow_from_dataframe(test, x_col = 'path', y_col  = None,
                                   class_mode = None,
                                   shuffle = False,
                                   target_size = (100,100) )

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB1

In [ ]:
model = Sequential()
model.add(EfficientNetB1(include_top = False, pooling = 'avg'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(12, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
             metrics = ['acc'])

In [ ]:
model.fit(train_gen, epochs = 3,validation_data = valid_gen)

In [ ]:
result = model.predict(test_gen, verbose = 1)

In [ ]:
result

In [ ]:
result2 = model.predict_classes(test_gen)
result2

In [ ]:
result2.shape

In [ ]:
train_gen.class_indices

In [ ]:
train['target'].unique()

In [ ]:
sorted(train['target'].unique())[1]

In [ ]:
class_list = []

for i , prediction in enumerate(result2):
    class_list.append(sorted(train['target'].unique())[prediction])

In [ ]:
class_list

In [ ]:
sub = pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')
sub

In [ ]:
sub['species'] = class_list
sub

In [ ]:
sub['file'] = test['path']
sub['file'] = sub['file'].apply(lambda x:x.split('/')[-1])
sub

In [ ]:
sub.to_csv('sub1.csv', index=0)